In [ ]:
# -*- coding: utf-8 -*-
"""Lista08.ipynb

In [ ]:
Automatically generated by Colaboratory.

In [ ]:
Original file is located at
    https://colab.research.google.com/drive/1gs92tf3LJsgLOr0hHgjY3OhuhgX45yzf

Lista 8 - Verossimilhança

 Introdução


<br>
# -*- coding: utf8<br>
from scipy import stats as ss<br>
import matplotlib.pyplot as plt<br>
from numpy.testing import assert_equal<br>
import numpy as np<br>
import pandas as pd<br>
plt.ion()<br>
import seaborn as sns<br>
PARAMETROS_RC = {  # Parâmetros RC para o matplotlib<br>
  'figure.figsize': (12, 8),<br>
  'axes.labelsize': 20,<br>
  'axes.titlesize': 20,<br>
  'legend.fontsize': 20,<br>
  'xtick.labelsize': 20,<br>
  'ytick.labelsize': 20,<br>
  'lines.linewidth': 4,<br>
}<br>
sns.set(<br>
    style="white",<br>
    palette="colorblind",<br>
    rc=PARAMETROS_RC,<br>
)<br>
def despine(ax=None):<br>
    if ax is None:<br>
        ax = plt.gca()<br>
    # Hide the right and top spines<br>
    ax.spines['right'].set_visible(False)<br>
    ax.spines['top'].set_visible(False)<br>
    # Only show ticks on the left and bottom spines<br>
    ax.yaxis.set_ticks_position('left')<br>
    ax.xaxis.set_ticks_position('bottom')<br>
ontinuando da aula passada. Vamos ver mais uma forma de entender um modelo de regressão linear. Lembre-se até agora falamos de correlação e covariância cobrindo os seguintes tópicos:

In [ ]:
1. Covariância
1. Coeficiente de Pearson (Covariância Normalizada)
1. Coeficiente de Pearson como sendo a fração do desvio de y capturado por x
1. Mínimos Quadrados

In [ ]:
Todos os passos acima chegam no mesmo local de traçar a "melhor" reta no gráfico de dispersão. Melhor aqui significa a reta que que minimiza o erro abaixo:

In [ ]:
$$\Theta = [\alpha, \beta]$$
$$L(\Theta) = \sum_i (y_i - \hat{y}_i)^2$$
$$L(\Theta) = \sum_i (y_i - (\beta x_i + \alpha))^2$$

In [ ]:
A solução que minimiza tal erro é:

In [ ]:
\begin{align}
 \alpha & = \bar{y} - \beta\,\bar{x}, \\[5pt]
  \beta &= \frac{ \sum_{i=1}^n (x_i - \bar{x})(y_i - \bar{y}) }{ \sum_{i=1}^n (x_i - \bar{x})^2 } \\[6pt]
            &= \frac{ \operatorname{Cov}(x, y) }{ \operatorname{Var}(x) } \\[5pt]
            &= r_{xy} \frac{s_y}{s_x}. \\[6pt]
\end{align}

 Visão probabílistica

In [ ]:
Vamos aprender uma última forma de pensar na regressão. Em particular, vamos fazer uso de uma visão probabílistica. Para tal, exploraremos o caso dos apartamentos de BH abaixo.

In [ ]:
Inicialmente, vamos observar os dados além do resultado da melhor regressão.
"""

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/pedroharaujo/ICD_Docencia/master/aptosBH.txt', index_col=0)
df['preco'] = df['preco'] / 1000

In [ ]:
plt.scatter(df['area'], df['preco'], edgecolors='k', s=80, alpha=0.6)

In [ ]:
plt.title('Preço de Apartamentos em BH')
plt.ylabel(r'Preço * $10^3$ (R\$)')
plt.xlabel(r'Área ($M^2$)')
despine()


O seaborn tem uma função regplot que plota a melhor reta além de um intervalo de confiança.
<br>
sns.regplot(x='area', y='preco', data=df, n_boot=10000,<br>
            line_kws={'color':'magenta', 'lw':4},<br>
            scatter_kws={'edgecolor':'k', 's':80, 'alpha':0.8})<br>
plt.title('Preco de Apartamentos em BH')<br>
plt.ylabel(r'Preço * $10^3$ (R\$)')<br>
plt.xlabel(r'Área ($M^2$)')<br>
despine()<br>
 reta pode ser recuperada usando scipy.


In [ ]:
model = ss.linregress(df['area'], df['preco'])
model


Usando esta reta podemos prever o preço de um apartamento usando apenas a área do mesmo.
<br>
beta = model.slope<br>
alpha = model.intercept<br>
novo_apt_area = 225<br>
preco = beta * novo_apt_area + alpha<br>
preco<br>
u seja, quando um apartamento de 225$m^2$ entra no mercado o mesmo custa em torno de 1M de reais.

 Erros Normais

In [ ]:
Agora, será que conseguimos chegar no mesmo pensando na regressão como um modelo probabilístico?

In [ ]:
Ao invés de focar no valor quadrático do erro, vamos pensar no caso ideal. Qual seria o valor ideal do erro da minha regressão? ZERO! Porém, isso é possível somente quando temos relações lineares perfeitas (ex: x = altura em cm, y = altura em polegadas), o que não é o caso geral.

In [ ]:
Nessa situação, vamos assumir que:
$$\epsilon_i \sim Normal(0, \sigma^2)$$

In [ ]:
Isto é, cada um dos erros vem de uma distribuição normal com média $0$. Como consequência, a média dos erros ainda vai ser uma normal centrada em $0$; e o desvio padrão dos erros poderá ser estimados através dos dados após a regressão.

 Verossimilhança

In [ ]:
Na regressão, queremos prever o valor de $y$ em função de $x$. No exemplo mostrado acima, queremos prever o preço de compra de um apartamento em BH em função da área.

In [ ]:
Estamos assumindo que:
$$\epsilon_i = y_i - \beta x_i - \alpha$$

In [ ]:
Ou seja, um erro na estimativa de um ponto não depende de outros.

In [ ]:
Temos uma função de densidade de probabilidade para os erros:
$$\epsilon_i \sim Normal(0, \sigma^2)$$

In [ ]:
Observe a função de densidade da Normal:
$$p(x|\mu, \sigma^2) = \frac{1}{\sqrt{\sigma^2 2 \pi}} e^{-\frac{(x-\mu)^2}{2 \sigma^2}}$$

In [ ]:
Logo:
$$p(\epsilon_i|\mu, \sigma^2) = \frac{1}{\sqrt{\sigma^2 2 \pi}} e^{-\frac{(\epsilon_i-\mu)^2}{2 \sigma^2}}$$

In [ ]:
Como no nosso modelo a média é zero, ficamos com:
$$p(\epsilon_i|\mu=0, \sigma^2) = \frac{1}{\sqrt{\sigma^2 2 \pi}} e^{-\frac{\epsilon_i^2}{2 \sigma^2}}$$
$$p(\epsilon_i|\sigma^2, \alpha, \beta) = \frac{1}{\sqrt{\sigma^2 2 \pi}} e^{-\frac{(y_i - \beta x_i - \alpha)^2}{2 \sigma^2}}$$

In [ ]:
A probabilidade mostrada acima é a probabilidade de uma observação (observar o erro $\epsilon_i$) dado os parâmetros $\sigma^2$, $\alpha$ e $\beta$!  
Isto é uma **verossimilhança**!

In [ ]:
O gráfico abaixo mostra a distribuição de probabilidade para erros vindos de uma distribuição normal com média igual a zero e desvio padrão igual a um.
"""

In [ ]:
x = np.linspace(-5, 5, 100)
plt.plot(x, ss.distributions.norm.pdf(x, scale=1))
plt.xlabel(r'$\epsilon_i$')
plt.ylabel(r'$p(\epsilon_i \mid \mu=0, \sigma=1)$')

In [ ]:
despine()


Um modelo estatístico nada mais é do que um conjunto de hipóteses que são supostas como válidas para a distribuição de probabilidades das variáveis aleatórias medidas na amostra. Em particular, estamos falando de modelos paramétricos, onde assumimos que o erro é uma normal com uma função de densidade conhecida (como mostrado no gráfico acima).<br>
Vamos focar numa base de 3 pontos. Além do mais, vamos assumir uma reta qualquer, dessa forma, o desvio padrão dos erros podem ser estimados através dos dados após a regressão. Se os erros vêm de uma distribuição normal com função de densidade já conhecida, qual dos três erros representados na figura abaixo é o mais verossímil?<br>
Observe:<br>
$$p(\epsilon_i|\sigma^2, \alpha, \beta) = \frac{1}{\sqrt{\sigma^2 2 \pi}} e^{-\frac{(y_i - \beta x_i - \alpha)^2}{2 \sigma^2}}$$<br>
Quanto menor o erro, maior a probabilidade, sendo assim, o segundo erro é o mais verossímil.<br>


In [ ]:
beta = 1
alpha = 1

In [ ]:
fig = plt.figure(figsize=(36, 10))

In [ ]:
x = np.array([2, 8, 5])
y = np.array([0, 1, 3])

In [ ]:
plt.subplot(121)
plt.scatter(x, y, edgecolors='k', s=80, alpha=0.6)
plt.title('3 Pontinhos')
plt.ylabel(r'Y')
plt.xlabel(r'X')

In [ ]:
y_bar = x * beta + alpha
plt.plot(x, y_bar, color='magenta')

In [ ]:
y_min = [min(y_i, y_bar_i) for y_i, y_bar_i in zip(y, y_bar)]
y_max = [max(y_i, y_bar_i) for y_i, y_bar_i in zip(y, y_bar)]
plt.vlines(x, ymin=y_min, ymax=y_max, color='magenta', lw=1)

In [ ]:
despine()

In [ ]:
plt.subplot(122)
plt.title('PDF da Normal')
ei_x = np.linspace(-10, 10, 100)
sigma = (y - y_bar).std(ddof=1)
plt.plot(ei_x, ss.distributions.norm.pdf(ei_x, scale=sigma))
plt.xlabel(r'$\epsilon_i$')
plt.ylabel(r'$p(\epsilon_i \mid \mu=0, \sigma={})$'.format(np.round(sigma, 2)))
despine()


Observe o exemplo considerando agora todos os pontos do exemplo de predição de preço de apartamentos em BH.
<br>
beta = 3.535719156333653<br>
alpha = 200.52361368989432<br>
fig = plt.figure(figsize=(36, 10))<br>
x = df['area']<br>
y = df['preco']<br>
plt.subplot(121)<br>
plt.scatter(x, y, edgecolors='k', s=80, alpha=0.6)<br>
plt.title('Preco de Apartamentos em BH')<br>
plt.ylabel(r'Preço * $10^3$ (R\$)')<br>
plt.xlabel(r'Área ($M^2$)')<br>
y_bar = x * beta + alpha<br>
plt.plot(x, y_bar, color='magenta')<br>
y_min = [min(y_i, y_bar_i) for y_i, y_bar_i in zip(y, y_bar)]<br>
y_max = [max(y_i, y_bar_i) for y_i, y_bar_i in zip(y, y_bar)]<br>
plt.vlines(x, ymin=y_min, ymax=y_max, color='magenta', lw=1)<br>
despine()<br>
plt.subplot(122)<br>
plt.title('PDF da Normal')<br>
ei_x = np.linspace(-1000, 1000, 100)<br>
sigma = (y - y_bar).std(ddof=1)<br>
plt.plot(ei_x, ss.distributions.norm.pdf(ei_x, scale=sigma))<br>
plt.xlabel(r'$\epsilon_i$')<br>
plt.ylabel(r'$p(\epsilon_i \mid \mu=0, \sigma={})$'.format(np.round(sigma, 2)))<br>
despine()<br>
# Erros Independentes

In [ ]:
Nos exemplos acima discutimos sobre qual erro é o mais verossímil... Mas como podemos calcular a verossimilhança de todos os erros?

In [ ]:
Ao assumir **independência**, a probabilidade total é obtida diretamente do produto das probabilidades. Logo:
$$p(E|\sigma^2, \alpha, \beta) = \prod_i p(\epsilon_i|\sigma^2, \alpha, \beta) = \prod_i p(\epsilon_i|\theta)$$
onde $E = \{e_1, e_2, ..., e_n\}$ e $\theta = \{\sigma^2, \alpha, \beta\}$.

In [ ]:
Acabamos de definir nossa primeira função de verossimilhança. Como podemos proceder agora? Maximizando a mesma! Ou seja, queremos os parâmetros ($\theta$) que melhor se ajustam os nossos dados.
Porém, maximizar produtórios é chato... O mundo das somas é mais bacana do que o mundo da multiplicação.

 Log-verossimilhança

In [ ]:
Como o $\log$ é uma função monotonicamente crescente, maximizar o $\log(f(x))$ é equivalente a maximizar $f(x)$. Portanto, podemos trabalhar apenas com o $\log$.

In [ ]:
Ao invés de maximizar a função de **verossimilhança** $L(\theta)$, vamos maximizar a função de **log-verossimilhança** $l(\theta)$:

In [ ]:
$$L(\theta) = \prod_i p(\epsilon_i|\theta)$$
$$\log(L(E|\theta)) = \log\left(\prod_i p(\epsilon_i|\theta)\right)$$
$$l(\epsilon_i|\theta) = \sum_i \log(p(\epsilon_i|\theta))$$
uma vez que $\log(AB) = \log(A) + \log(B)$.

In [ ]:
Substituindo os parâmetros $\sigma^2$, $\alpha$, $\beta$ nas funções e fazendo algumas manipulações, temos que a verossimilhança e a log-verossimilhança são dadas por:

In [ ]:
$$L(E|\theta) = \prod_i \frac{1}{\sqrt{\sigma^2 2 \pi}} e^{-\frac{(y_i - \beta x_i - \alpha)^2}{2 \sigma^2}}$$

In [ ]:
$$l(E|\theta) = -n\log(\sqrt{2\pi}) -n\log(\sigma) - \frac{\sum_i (y_i - \beta x_i - \alpha)^2}{2 \sigma^2}$$

In [ ]:
* A partir da fórmula da verossimilhança, dada por:
$$L(E|\theta) = \prod_i \frac{1}{\sqrt{\sigma^2 2 \pi}} e^{-\frac{(y_i - \beta x_i - \alpha)^2}{2 \sigma^2}}$$
verifique que a expressão da log-verossimilhança é dada por:
$$l(E|\theta) = -n\log(\sqrt{2\pi}) -n\log(\sigma) - \frac{\sum_i (y_i - \beta x_i - \alpha)^2}{2 \sigma^2}$$

In [ ]:
$$L(E|\theta) = \prod_i \frac{1}{\sqrt{\sigma^2 2 \pi}} e^{-\frac{(y_i - \beta x_i - \alpha)^2}{2 \sigma^2}}$$
$$\log(L(E|\theta)) = \log\left(\prod_i \frac{1}{\sqrt{\sigma^2 2 \pi}} e^{-\frac{(y_i - \beta x_i - \alpha)^2}{2 \sigma^2}}\right)$$
$$l(\epsilon_i|\theta) = \sum_i \log\left(\frac{1}{\sqrt{\sigma^2 2 \pi}} e^{-\frac{(y_i - \beta x_i - \alpha)^2}{2 \sigma^2}}\right)$$
uma vez que $log(AB) = log(A) + log(B)$.
$$l(\epsilon_i|\theta) = \sum_i \log\left(\frac{1}{\sqrt{\sigma^2 2 \pi}}\right) + \sum_i log\left(e^{-\frac{(y_i - \beta x_i - \alpha)^2}{2 \sigma^2}}\right)$$
$$l(\epsilon_i|\theta) = n \log\left(\frac{1}{\sqrt{\sigma^2 2 \pi}}\right) + \sum_i \log\left(e^{-\frac{(y_i - \beta x_i - \alpha)^2}{2 \sigma^2}}\right)$$
$$l(\epsilon_i|\theta) = n \log\left(\frac{1}{\sqrt{\sigma^2 2 \pi}}\right) - \frac{\sum_i (y_i - \beta x_i - \alpha)^2}{2 \sigma^2}$$
uma vez que $\log(A^x) = x \log(A)$ e $\log(e) = 1$.
$$l(\epsilon_i|\theta) = - n \log\left(\sqrt{\sigma^2 2 \pi}\right) - \frac{\sum_i (y_i - \beta x_i - \alpha)^2}{2 \sigma^2}$$
uma vez que $\log(\frac{1}{x}) = - \log(x)$.
$$l(\epsilon_i|\theta) = - n \log(\sqrt{2 \pi}) - n \log(\sqrt{\sigma}) - \frac{\sum_i (y_i - \beta x_i - \alpha)^2}{2 \sigma^2}$$

In [ ]:
Maximizar a log-verossimilhança implica em achar os parâmetros que melhor se ajustam aos dados. Além disso, lembre-se que maximizar é achar o local onde a derivada é zero.

In [ ]:
Para isso, devemos resolver as derivadas em relação a cada um dos parâmetros: $\sigma$, $\alpha$ e $\beta$.

In [ ]:
$$\alpha = \bar{y} - \beta \bar{x}$$
$$\beta = \frac{ \sum_{i=1}^n (x_i - \bar{x})(y_i - \bar{y}) }{ \sum_{i=1}^n (x_i - \bar{x})^2 }$$
$$\sigma^2 = \frac{\sum_i (y_i - \beta x_i - \alpha)^2}{n}$$

In [ ]:
Observe que ao maximizar $\sigma$, chegamos no estimador da variância. Além disso, o $\sigma$ não impacta os valores de $\alpha$ e $\beta$, mas é útil para entendermos os erros do estimador (é a variância dos erros afinal).

In [ ]:
Por fim, note que maximizar a verossimilhança da regressão é o mesmo que minimizar os erros quadrados.

 Estimação da Máxima Verossimilhança e Mínimos Quadrados

In [ ]:
Por que escolher os mínimos quadrados? Uma justificativa envolve a estimativa de máxima verossimilhança.

In [ ]:
Imagine que temos uma amostra de dados $v_1, \cdots, v_n$ que vem de uma distribuição que depende de algum parâmetro desconhecido $\theta$:

In [ ]:
$$p(v_1, \cdots, v_n~|~\theta)$$

In [ ]:
Se não conhecêssemos theta, poderíamos nos virar e pensar nessa quantidade como a probabilidade de $\theta$ dada a amostra:

In [ ]:
$$L(\theta~|~v_1, \cdots, v_n)$$

In [ ]:
Sob essa abordagem, o mais provável $\theta$ é o valor que maximiza essa função de verossimilhança; isto é, o valor que torna os dados observados os mais prováveis. Podemos fazer a mesma coisa no caso de uma distribuição contínua, na qual temos uma função de distribuição de probabilidade e não uma função de massa de probabilidade.

In [ ]:
De volta à regressão. Uma suposição que muitas vezes é feita sobre o modelo de regressão simples é que os erros de regressão são normalmente distribuídos com média $0$ e algum desvio padrão (conhecido) $\sigma$. Se esse for o caso, a probabilidade de ver um par $(x_i, y_i)$ é:

In [ ]:
$$L(\alpha, \beta~|~x_i, y_i, \sigma) = \frac{1}{\sqrt{2\pi\sigma}}\exp{\left(\frac{-(y_i-\alpha-\beta x_i)^2}{2\sigma^2}\right)}$$

In [ ]:
A probabilidade de todo o conjunto de dados é o produto das probabilidades individuais, que é maior precisamente quando alfa e beta são escolhidos para minimizar a soma dos erros quadrados. Ou seja, nesse caso (e com essas suposições), minimizar a soma dos erros quadrados é equivalente a maximizar a probabilidade dos dados observados.

 Qualidade da regressão

In [ ]:
Após encontrar os parâmetros do modelo que melhor se ajustam aos dados (minimizam os erros quadrados, maximizam a verossimilhança), devemos avaliar a qualidade do ajuste. A qualidade da regressão pode ser analisada através de gráficos ou métricas, como o R-quadrado, estudado anteriormente.

 Gráfico residual

In [ ]:
O gráfico residual é uma forma de visualizar os erros versus a variável preditora para verificar a pressuposição de que os erros são aleatoriamente distribuídos e têm variância constante. De maneira ideal, os pontos devem cair aleatoriamente em ambos os lados de 0, sem padrões reconhecíveis nos pontos. Como os erros são centrados em zero, ao plotar valor de x pelo erro queremos pontos igualmente dispersados  positivos e negativos.
Lembre-se da PDF da distribuição normal. A mediana e a média são iguais, ou seja, 50% dos erros se concentram acima de 0 e 50% abaixo.

In [ ]:
Observe o gráfico residual para o exemplo de prever o preço de um apartamento usando apenas a área do mesmo. No eixo-x temos a área do aparatamento e no eixo-y o erro. O gráfico não parece ter um padrão (é o que queremos) já que os erros estão espalhados em ambos os lados da linha tracejada que marca o erro igual a 0. Entretanto, os erros aparentam ser piores quando a área dos apartamentos (variável x) aumenta.
"""

In [ ]:
sns.residplot(x='area', y='preco', data=df,
              line_kws={'color':'magenta', 'lw':4},
              scatter_kws={'edgecolor':'k', 's':80, 'alpha':0.8})
plt.ylabel(r'$\epsilon_i$')
plt.xlabel(r'Área ($M^2$)')
despine()


## QQ Plot<br>
Por fim, outra forma de verificar se o modelo é bom se chama QQ Plot.<br>
Esse gráfico é útil para checar a adequação da distribuição de frequência dos dados à uma distribuição de probabilidades. No caso dos modelos de regressão, o QQ Plot é usado para verificar se os erros apresentam distribuição normal.<br>
Uma forma de construir o QQ Plot consiste em ordenar os erros (eixo-y), comparando com o local esperado do mesmo no modelo (distribuição normal). Nesse caso, a mediana fica no centro do plot. O erro mediano é zero caso a normal seja verdadeira!<br>
Outra forma de construir o QQ Plot consiste em pegar o valor z-norm que leva para a probabilidade de cada um dos erros ordenados no modelo e plotar os erros ordenados no eixo-y e os valores z-normalizados no eixo-x. O gráfico abaixo apresenta o QQ Plot do exemplo de prever o preço de um apartamento usando apenas a área do mesmo.<br>
Idealmente quanto mais próximo de uma reta melhor, uma vez que a reta quer dizer que os erros são perfeitamente normais. Sendo assim, na regressão perfeita observamos uma linha reta no QQ plot.<br>


In [ ]:
ss.probplot(y - y_bar, plot=plt.gca());


## Exercícios - Outros datasets<br>
### Enunciado<br>
Abaixo estão listados algumas conjuntos de dados contendo relações entre variáveis:<br>
**Supernovas**: Atualmente, uma das teorias mais aceitas sobre a formação do universo, diz que o universo está em constante expansão. Supernovas são estrelas que explodiram e morreram recentemente. Os dados contêm registros dessas supernovas. Cada linha na tabela corresponde a uma supernova próxima da Terra observada por astrônomos, indicando o quão longe da Terra a supernova estava e o quão rápido ela se afastava.<br>
**Stocks and Unemployment**: A Bolsa está diretamente ligada à economia do país e do mundo. Quando ela desaba, isso pode ter consequências negativas até no dia a dia de quem nem sabe como a Bolsa funciona. Por exemplo, o desemprego pode aumentar e a inflação acelerar, tornando os produtos no supermercado mais caros. Com base nisso, os dados apresentam a evolução no preço de uma ação e a taxa de desemprego em determinado período.<br>
**Stocks and Interest**: Além de relacionar o valor de ações na Bolsa com o índice de desemprego, é possível correlacionar o valor das ações e outros indicadores. Os dados apresentam a evolução no preço de uma ação e a taxa de juros em determinado período.<br>
**Dugongs**: Os dados incluem informações de peixe-bois. Cada linha contém o comprimento e a idade de indivíduos. A base de dados é muito utilizada para fazer predições do comprimento desses animais de acordo com a idade.<br>
- **Exercício:** Para cada um dos datasets você deverá:<br>
    1. Carregar os dados do arquivo.  <br>
    2. Fazer uma regressão linear.  <br>
    3. Avaliar a qualidade do modelo através do cálculo do $R^2$ e dos gráficos de erros: Gráfico residual e QQ Plot.  <br>
    4. Verificar se os erros são independentes. Para isso, conte quantos erros são maiores e quantos são menores do que zero.<br>
Antes de realizar o exercício, é útil você definir as seguintes funções com base nos parâmetros citados em cada uma:<br>
Note que após definir as funções pedidas corretamente até o fim do primeiro dataset, os demais irão executar sem erros, não precisando ser alterados.<br>


In [ ]:
def error(alpha, beta, x, y):
    return y - (beta * x + alpha)

In [ ]:
def sum_of_squared_errors(alpha, beta, x, y):
    return sum(error(alpha, beta, x, y) ** 2)

In [ ]:
def total_sum_of_squares(y):
    yMean = y.mean()
    return sum((y - yMean) ** 2)

In [ ]:
def r_squared(alpha, beta, x, y):
    sse = sum_of_squared_errors(alpha, beta, x, y)
    Tsse = total_sum_of_squares(y)
    return 1 - sse / Tsse

In [ ]:
def test(alpha, beta, x, y):
    sq_res = sum((y - (beta * x + alpha)) ** 2)
    y_mean = y.mean()
    sq_tot = sum((y - y_mean) ** 2)
    return 1 - sq_res / sq_tot


### 1. Supernovas<br>
#### 1.1 Carregar os dados<br>


In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/pedroharaujo/ICD_Docencia/master/close_novas.csv')
x = df['Distance (million parsecs)']
y = df['Speed (parsecs/year)']


#### 1.2 Regressão Linear
<br>
def linear_regression(x, y):<br>
    xMean = x.mean()<br>
    yMean = y.mean()<br>
    cov = np.cov(x, y)<br>
    beta = cov[1][0] / cov[0][0]<br>
    alpha = yMean - beta * xMean<br>
    return alpha, beta<br>
(a1, b1) = linear_regression(x,y)<br>
assert_equal(round(a1, 6), 0.000167)<br>
assert_equal(round(b1, 6), 0.000068)<br>
### 1.3 Avaliação do Modelo via R²


<br>
# avaliacao do Rsquared<br>
r2 = r_squared(a1, b1, x, y)<br>
assert_equal(round(r2, 4), 0.9645)<br>
sns.residplot(x='Distance (million parsecs)', y='Speed (parsecs/year)', data=df, line_kws={'color':'magenta', 'lw':4}, scatter_kws={'edgecolor':'k', 's':80, 'alpha':0.8})<br>
plt.ylabel(r'$\epsilon_i$')<br>
despine()<br>
plt.show()<br>
# GABARITO GRAFICO<br>
# sns.residplot(x='Distance (million parsecs)', y='Speed (parsecs/year)', data=df,<br>
#               line_kws={'color':'magenta', 'lw':4},<br>
#               scatter_kws={'edgecolor':'k', 's':80, 'alpha':0.8})<br>
# plt.ylabel(r'$\epsilon_i$')<br>
# despine()<br>
# plt.show()<br>
### 1.4 Verificando se os erros são independentes


In [ ]:
def error_verification(alpha, beta, x, y):
    neg: int = 0
    pos: int = 0
    for (i, j) in zip(x, y):
        if error(alpha, beta, i, j) < 0:
            neg = neg + 1
        else:
            pos = pos + 1
    return (pos, neg)

In [ ]:
(maiores, menores) = error_verification(a1, b1, x, y)
assert_equal(maiores, 78)
assert_equal(menores, 78)

In [ ]:
ss.probplot(y - y.mean(), plot=plt)
despine()
plt.show()

ss.probplot(y - y.mean(), plot=plt)<br>
despine()<br>
plt.show()


### 2. Stocks anad Unemployment<br>
#### 2.1 Carregar os dados<br>


In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/pedroharaujo/ICD_Docencia/master/stocks_unemployment.csv')
x = df['Unemployment_Rate']
y = df['Stock_Index_Price']


#### 2.2 Regressão Linear
<br>
(a2, b2) = linear_regression(x,y)<br>
assert_equal(round(a2, 6), 4471.339321)<br>
assert_equal(round(b2, 6), -588.962076)<br>
### 2.3 Avaliação do Modelo via R²


<br>
# avaliacao do Rsquared<br>
r2 = r_squared(a2, b2, x, y)<br>
assert_equal(round(r2, 4), 0.8507)<br>
sns.residplot(x='Unemployment_Rate', y='Stock_Index_Price', data=df, line_kws={'color':'magenta', 'lw':4}, scatter_kws={'edgecolor':'k', 's':80, 'alpha':0.8})<br>
plt.ylabel(r'$\epsilon_i$')<br>
despine()<br>
plt.show()<br>
# GABARITO GRAFICO<br>
# sns.residplot(x='Unemployment_Rate', y='Stock_Index_Price', data=df,<br>
#               line_kws={'color':'magenta', 'lw':4},<br>
#               scatter_kws={'edgecolor':'k', 's':80, 'alpha':0.8})<br>
# plt.ylabel(r'$\epsilon_i$')<br>
# despine()<br>
# plt.show()<br>
### 2.4 Verificando se os erros são independentes


In [ ]:
(maiores, menores) = error_verification(a2, b2, x, y)
assert_equal(maiores, 13)
assert_equal(menores, 11)

In [ ]:
ss.probplot(y - y.mean(), plot=plt)
despine()
plt.show()

ss.probplot(y - y.mean(), plot=plt)<br>
despine()<br>
plt.show()


### 3. Stocks and Interest<br>
#### 3.1 Carregar os dados<br>


In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/pedroharaujo/ICD_Docencia/master/stocks_interest.csv')
x = df['Interest_Rate']
y = df['Stock_Index_Price']


#### 3.2 Regressão Linear
<br>
(a3, b3) = linear_regression(x,y)<br>
assert_equal(round(a3, 6), -99.464319)<br>
assert_equal(round(b3, 6), 564.203892)<br>
### 3.3 Avaliação do Modelo via R²


<br>
# avaliacao do Rsquared<br>
r2 = r_squared(a3, b3, x, y)<br>
assert_equal(round(r2, 4), 0.8757)<br>
sns.residplot(x='Interest_Rate', y='Stock_Index_Price', data=df, line_kws={'color':'magenta', 'lw':4}, scatter_kws={'edgecolor':'k', 's':80, 'alpha':0.8})<br>
plt.ylabel(r'$\epsilon_i$')<br>
despine()<br>
plt.show()<br>
# GABARITO GRAFICO<br>
# sns.residplot(x='Interest_Rate', y='Stock_Index_Price', data=df,<br>
#               line_kws={'color':'magenta', 'lw':4},<br>
#               scatter_kws={'edgecolor':'k', 's':80, 'alpha':0.8})<br>
# plt.ylabel(r'$\epsilon_i$')<br>
# despine()<br>
# plt.show()<br>
### 3.4 Verificando se os erros são independentes


In [ ]:
(maiores, menores) = error_verification(a3, b3, x, y)
assert_equal(maiores, 12)
assert_equal(menores, 12)

In [ ]:
ss.probplot(y - y.mean(), plot=plt)
despine()
plt.show()

ss.probplot(y - y.mean(), plot=plt)<br>
despine()<br>
plt.show()


### 4. Dugongs<br>
#### 4.1 Carregar os dados<br>


In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/pedroharaujo/ICD_Docencia/master/dugongs.csv')
x = df['Age']
y = df['Length']


#### 4.2 Regressão Linear
<br>
(a4, b4) = linear_regression(x,y)<br>
assert_equal(round(a4, 6), 2.018286)<br>
assert_equal(round(b4, 6), 0.028955)<br>
### 4.3 Avaliação do Modelo via R²


<br>
# avaliacao do Rsquared<br>
r2 = r_squared(a4, b4, x, y)<br>
assert_equal(round(r2, 4), 0.6883)<br>
sns.residplot(x='Age', y='Length', data=df, line_kws={'color':'magenta', 'lw':4}, scatter_kws={'edgecolor':'k', 's':80, 'alpha':0.8})<br>
plt.ylabel(r'$\epsilon_i$')<br>
despine()<br>
plt.show()<br>
# GABARITO GRAFICO<br>
# sns.residplot(x='Age', y='Length', data=df,<br>
#               line_kws={'color':'magenta', 'lw':4},<br>
#               scatter_kws={'edgecolor':'k', 's':80, 'alpha':0.8})<br>
# plt.ylabel(r'$\epsilon_i$')<br>
# despine()<br>
# plt.show()<br>
### 4.4 Verificando se os erros são independentes


In [ ]:
(maiores, menores) = error_verification(a4, b4, x, y)
assert_equal(maiores, 16)
assert_equal(menores, 11)

In [ ]:
ss.probplot(y - y.mean(), plot=plt)
despine()
plt.show()

ss.probplot(y - y.mean(), plot=plt)<br>
despine()<br>
plt.show()


## Estimador de Máxima Verossimilhança<br>
Conforme vimos, a verossimilhança representa a probabilidade dos nossos dados $X$ (ou erros $E$) condicionados em um modelo. O modelo aqui são os parâmetros $\theta$ e a função de probabilidade $p$.<br>
$$X = \{x_1, x_2, ..., x_n\}$$<br>
$$p(X|\theta) = \prod_i p(x_i|\theta)$$<br>
No modelo que estudamos:<br>
$\epsilon_i \sim Normal(0, \sigma^2)$  <br>
$p(\epsilon_i|\sigma^2, \alpha, \beta) = \frac{1}{\sqrt{\sigma^2 2 \pi}} e^{-\frac{(y_i - \beta x_i - \alpha)^2}{2 \sigma^2}}$  <br>
$E = \{e_1, e_2, ..., e_n\}$ e $\theta = \{\sigma^2, \alpha, \beta\}$  <br>
$p(E|\theta) = \prod_i p(\epsilon_i|\theta)$<br>
Entretanto, podemos estimar os parâmetros de qualquer modelo probabilístico.<br>
Imagine que observamos variáveis categóricas $X$. $X$ corresponde ao efeito do remédio em um grupo de pessoas, onde cada $x_i$ assume valo igual a 1, caso o remédio tenha funcionado e 0 caso contrário. Por exemplo: $X = \{1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0\}$<br>
Podemos modelar estes dados a partir de uma distribuição [Bernoulli](https://en.wikipedia.org/wiki/Bernoulli_distribution). Ou seja,  $x_i \sim Bernoulli(\theta)$ onde $\theta$ representa a probabilidade do remédio funcionar para o paciente (probabilidade de sucesso).<br>
* Derive o estimador de máxima verossimilhança para a distribuição Bernoulli.<br>
$$x_i \sim \text{Bernoulli}(\theta)$$<br>
$$p(x_i|\theta) = \theta^{x_i} (1-\theta)^{(1 - x_i)}$$<br>
Estimador de máxima verossimilhança:<br>
$$L(X|\theta) = \prod_i p(x_i|\theta) = \prod_i \theta^{x_i} (1-\theta)^{(1 - x_i)} = \theta^{\sum_i x_i} (1-\theta)^{\sum_i (1 - x_i)}$$<br>
lembrando que $a^x b^y * a^z b^w = a^{x+z} b^{y+w}$<br>
Estimador de máxima log-verossimilhança:<br>
$$l(X|\theta) = \log\left(\prod_i p(x_i|\theta)\right) = \sum_i \log\left(\theta^{x_i} (1-\theta)^{(1 - x_i)}\right) = \sum_i x_i \log(\theta) + \sum_i (1-x_i) \log(1-\theta)$$<br>
lembrando que $\log(ab) = \log(a) + \log(b)$ e $\log(a^x) = x\log(a)$<br>
* Após derivar o estimador de máxima verossimilhança para a distribuição Bernoulli encontre o valor de $\theta$ que maximiza a verossimilhança.<br>
Assumimos que as observações são independentes, maximizar a log da verossimilhança corresponde a derivar essa função e igualar a zero, encontrando o seu ponto de máximo.<br>
Calculando a derivada, temos:<br>
$$\frac{\partial l(X|\theta)}{\theta} = \frac{\partial (\sum_i x_i \log(\theta) + \sum_i (1-x_i) \log(1-\theta))}{\partial \theta} = \frac{\sum_i x_i}{\theta} - \frac{\sum_i (1-x_i)}{(1-\theta)}$$<br>
Lembrando que $\frac{\partial log(a)}{\partial a} = \frac{1}{a}$ e $\frac{\partial log(1-a)}{\partial a} = \frac{1}{(1-a)}$<br>
Igualando a derivada a zero, encontramos o valor de $\theta$ que maximiza a log-verossimilhança:<br>
$$\frac{\sum_i x_i}{\theta} - \frac{\sum_i (1-x_i)}{(1-\theta)} = 0$$<br>
$$(1-\theta) \sum_i x_i = \theta \sum_i (1-x_i)$$<br>
$$\sum_i x_i - \theta \sum_i x_i = n\theta - \theta \sum_i x_i$$<br>
$$\sum_i x_i = n\theta$$<br>
$$\theta = \frac{\sum_i x_i}{n} = \bar x$$<br>
